In [1]:
from tqdm import tqdm
import torch
import torchvision as tv

from mycv.paths import WEIGHTS_DIR
from mycv.datasets.imagenet import imagenet_val

In [2]:
model = tv.models.resnet101(pretrained=False)
model = model.cuda()
weights = WEIGHTS_DIR / 'resnet101-5d3b4d8f.pth'
model.load_state_dict(torch.load(weights))

<All keys matched successfully>

In [3]:
def benchmark(model, batch_size, img_size, channels_last=False, iterations=128, half=False):
    if channels_last:
        model.to(memory_format=torch.channels_last)
    else:
        model.to(memory_format=torch.contiguous_format)
    if half:
        model.half()
    
    for _ in tqdm(range(iterations)):
        x = torch.rand(batch_size, 3, img_size, img_size)
        if channels_last:
            x = x.to(memory_format=torch.channels_last)
        if half:
            x = x.half()
        x = x.cuda()
        x = model(x)

In [4]:
# standard inference
model.eval()
with torch.no_grad():
    benchmark(model, 64, 256, channels_last=False, iterations=3)
    benchmark(model, 64, 256, channels_last=False, iterations=128)
    print('')
    benchmark(model, 64, 256, channels_last=True, iterations=3)
    benchmark(model, 64, 256, channels_last=True, iterations=128)

 33%|███▎      | 1/3 [00:00<00:00,  6.66it/s]

100%|██████████| 128/128 [00:25<00:00,  5.10it/s]


In [5]:
# half precision inference
model.eval()
with torch.no_grad():
    benchmark(model, 64, 256, channels_last=False, iterations=3, half=True)
    benchmark(model, 64, 256, channels_last=False, iterations=128, half=True)
    print('')
    benchmark(model, 64, 256, channels_last=True, iterations=3, half=True)
    benchmark(model, 64, 256, channels_last=True, iterations=128, half=True)

 33%|███▎      | 1/3 [00:00<00:00,  6.91it/s]

100%|██████████| 128/128 [00:15<00:00,  8.12it/s]


In [6]:
# standard training

In [7]:
# amp training

In [8]:
torch.backends.cudnn.version()

8004